In [ ]:
!pip install insightface onnxruntime-gpu gdown

In [ ]:
from pathlib import Path
from torch.utils.data import DataLoader, Subset
import requests
import zipfile
import gdown

ROOT = Path("./data")
if not ROOT.exists():
    ROOT.mkdir()
    
DATASET_PATH = ROOT / "vggface2"
if not DATASET_PATH.exists():
    DATASET_PATH.mkdir()
    gdown.download(id="1dyVQ7X3d28eAcjV3s3o0MT-HyODp_v3R", output=str(DATASET_PATH / "vgg.zip"))
    tmp = DATASET_PATH / "vgg.zip"
    with zipfile.ZipFile(str(tmp)) as zip_ref:
        zip_ref.extractall(str(DATASET_PATH))
    tmp.unlink()

In [ ]:
from threading import Lock
from time import monotonic
from insightface.app import FaceAnalysis
import torch
import cv2
from concurrent.futures import ThreadPoolExecutor

CUDA_AVAILABLE = torch.cuda.is_available()
NUM_WORKERS = 1
LOCK = Lock()
CNT = 0

face_apps: list[FaceAnalysis] = []
for i in range(NUM_WORKERS):
    print(f"Preparing {i}")
    app = FaceAnalysis(name='buffalo_l', providers=[("CUDAExecutionProvider", {"device_id": i % 2})])  # Use 'CUDAExecutionProvider' for GPU
    app.prepare(ctx_id=(-1 if not CUDA_AVAILABLE else i % 2))  # ctx_id=-1 for CPU, 0 for GPU
    print(f"Prepared {i}")
    face_apps.append(app)

def run(img: str):
    global CNT
    with LOCK:
        app = face_apps[CNT]
        CNT = (CNT + 1) % NUM_WORKERS

    return app.get(cv2.imread(img))

pool = ThreadPoolExecutor(NUM_WORKERS)

In [ ]:
from torch import nn

class MetricsNet(nn.Module):
    def __init__(self, d: int):
        super().__init__()
        
        self.seq = nn.Sequential([
            nn.Linear(d, d),
            nn.ReLU(),
            nn.Linear(d, d//2),
            nn.ReLU(),
            nn.Linear(d//2, d//4),
            nn.ReLU(),
            nn.Linear(d//4, d//8),
            nn.ReLU(),
            nn.Linear(d//8, d//16),
            nn.ReLU(),
            nn.Linear(d//16, d//32),
            nn.ReLU(),
            nn.Linear(d//32, d//64),
        ])
        
    def forward(self, x):
        return self.seq(x)


# 1. Definir Neural Network Metrics+
# 2. Treinar localmente...
# 3. Treinar no kaggle, salvar checkpoints.
# 4. treinar ista-net com base nisso...
# 5. 

In [18]:
import cv2
import torch
from insightface.app import FaceAnalysis

app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])  # Use 'CUDAExecutionProvider' for GPU
app.prepare(-1)
emb2=app.get(cv2.imread("./data/vggface2/train/n000002/0001_01.jpg"))[0].embedding

type(emb2)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/erb/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/erb/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/erb/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/erb/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/erb/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (64

/home/erb/ufsc/INE5448/.venv/lib/python3.12/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


numpy.ndarray

In [ ]:
from torchvision.datasets import ImageFolder
import cv2
import torch
from random import Random

def get_class_to_idx_list(dataset: ImageFolder) -> list[list[int]]:
    ret = []
    cur = []
    last = dataset.targets[0]
    for i, s in enumerate(dataset.targets):
        if s != last:
            last = s
            ret.append(cur)
            cur = []
        cur.append(i)
    ret.append(cur)
    return ret


def generate_batches(dataset: ImageFolder, generator: Random, batch_size: int):
    class_to_idx_list = get_class_to_idx_list(dataset)
    classes = list(range(len(class_to_idx_list)))
    while True:
        batch = []
        for i in range(batch_size//2):
            same_img_class = generator.choice(classes)
            same_img1_idx, same_img2_idx = generator.choices(class_to_idx_list[same_img_class], k=2)
            
            batch.append((dataset[same_img1_idx], dataset[same_img2_idx], True))

            diff_img_class1, diff_img_class2 = generator.choices(classes, k=2)
            diff_img1_idx = generator.choice(class_to_idx_list[diff_img_class1])
            diff_img2_idx = generator.choice(class_to_idx_list[diff_img_class2])
            
            batch.append((dataset[diff_img1_idx], dataset[diff_img2_idx], False))
        yield batch

dataset = ImageFolder("./data/vggface2/train/", loader=lambda pth: torch.from_numpy(cv2.imread(str(pth))))
generator = Random("abcd")

[315,
 205,
 387,
 229,
 485,
 259,
 273,
 156,
 380,
 382,
 314,
 288,
 248,
 449,
 319,
 283,
 374,
 450,
 286,
 396,
 398,
 451,
 309,
 373,
 382,
 264,
 268,
 305,
 519,
 357,
 306,
 237,
 374,
 206,
 419,
 376,
 391,
 265,
 493,
 364,
 246,
 411,
 330,
 216,
 440,
 400,
 327,
 212,
 381,
 438,
 450,
 304,
 365,
 444,
 346,
 495,
 376,
 321,
 437,
 378,
 394,
 339,
 511,
 304,
 398,
 422,
 320,
 306,
 320,
 259,
 192,
 304,
 319,
 448,
 594,
 536,
 498,
 448,
 454,
 438,
 434,
 386,
 303,
 469,
 206,
 250,
 176,
 432,
 216,
 425,
 603,
 482,
 346,
 485,
 285,
 375,
 572,
 445,
 515,
 226,
 401,
 478,
 359,
 511,
 283,
 217,
 336,
 301,
 440,
 371,
 392,
 207,
 417,
 349,
 365,
 316,
 310,
 186,
 429,
 271,
 207,
 372,
 358,
 334,
 285,
 404,
 158,
 266,
 226,
 304,
 432,
 446,
 367,
 437,
 258,
 314,
 405,
 312,
 446,
 433,
 392,
 487,
 291,
 358,
 408,
 373,
 494,
 385,
 379,
 292,
 542,
 375,
 289,
 400,
 417,
 289,
 328,
 384,
 459,
 484,
 286,
 280,
 351,
 560,
 266,
 388,
 330